In [1]:
%cd /content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1/

/content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1


In [38]:
!python run.py

Start Training...
Start Testing...


In [2]:
import numpy as np
import data_utils
import implementations
%load_ext autoreload
%autoreload 2

In [3]:
train_data, train_label = data_utils.load_data('Data/train.csv')
train_data.shape, train_label.shape

((250000, 30), (250000,))

In [4]:
def cross_validation(train_data, train_label, k_fold, eval_func, input_dict, eval_func_name):
  data_num = train_data.shape[0]
  data_dim = train_data.shape[1]
  fold_part_idx = data_utils.k_fold(data_num, k_fold)

  loss_list = []
  acc_list = []
  for k in range(k_fold): 
    cur_train_idx = np.concatenate([fold_part_idx[:k, :], fold_part_idx[k+1:, :]], axis = 0)
    cur_train_idx = cur_train_idx.flatten()
    cur_test_idx = fold_part_idx[k, :]
    x_tr = train_data[cur_train_idx, :]
    x_te = train_data[cur_test_idx, :]
    y_tr = train_label[cur_train_idx]
    y_te = train_label[cur_test_idx]
    if(eval_func_name == 'ls'):
      w, loss = eval_func(y_tr, x_tr)
    elif(eval_func_name == 'ridge'):
      w, loss = eval_func(y_tr, x_tr, input_dict.get('lambda_'))
    elif(eval_func_name == 'logistic'):
      w, loss = eval_func(y_tr, x_tr, initial_w = input_dict.get('initial_w'), max_iters = input_dict.get('max_iters'), gamma = input_dict.get('gamma'))
    elif(eval_func_name == 'reg_logistic'):
      w, loss = eval_func(y_tr, x_tr, lambda_ = input_dict.get('lambda_'), initial_w = input_dict.get('initial_w'), max_iters = input_dict.get('max_iters'), gamma = input_dict.get('gamma'))
    _, train_acc = implementations.compute_statistics_all(y_tr, x_tr, w, func_type = input_dict['func_type'])
    test_loss, test_acc = implementations.compute_statistics_all(y_te, x_te, w, func_type = input_dict['func_type'])
    loss_list.append(test_loss)
    acc_list.append(test_acc)
    # print(test_acc)
  return np.mean(loss_list), np.mean(acc_list)

In [33]:
k_fold_num = 5
group_num = 1

In [34]:
max_acc = 0
lambdas = np.logspace(-10, 0, 20)
eval_func_name = 'ridge'
test_func = implementations.ridge_regression
degrees = list(range(5,10))
n_sigma = 0.0
for clean in [1]:
  for poly in [1]:
    if(poly):
      for degree in degrees:
        degree_list = [1] * 3
        degree_list[group_num] = degree
        for normalize in [1,0]:
          for all_poly in [True]:
            for add_bias in [True, False]:
              for add_log in [False]:
                add_log_list = [False] * 3
                add_log_list[group_num] = add_log
                group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                              data_utils.process_data(train_data, train_label, clean = clean)
                group0_x, group1_x, group2_x = \
                        data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly, add_log_list)
                cur_group = [group0_x, group1_x, group2_x]
                cur_labels = [group0_labels, group1_labels, group2_labels]
                group_x = cur_group[group_num]
                group_label = cur_labels[group_num]
                if(normalize):
                  group_x = data_utils.normalize(group_x)
                if(n_sigma > 0):
                  group_x = data_utils.handle_outlier(group_x, n = n_sigma)
                if(add_bias):
                  group_x = data_utils.add_bias(group_x)
                for lambda_ in lambdas:
                  input_dict = {'lambda_':lambda_,\
                          'initial_w':np.zeros(group_x.shape[1]),\
                          'max_iters':10000,\
                          'gamma':0.001,\
                          'sgd':False,\
                          'func_type':'linear'}
                  loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict, eval_func_name)
                  if(acc > max_acc):
                    print(f'Found a better scheme.\
                        Clean:{clean},\
                        Poly:{poly},\
                        Degree:{degree},\
                        lambda:{lambda_}, \
                        Normalize: {normalize},\
                        bias: {add_bias},\
                        all_poly:{all_poly},\
                        add_log:{add_log},\
                        Acc:{acc}')
                    max_acc = acc

Found a better scheme.                        Clean:1,                        Poly:1,                        Degree:5,                        lambda:1e-10,                         Normalize: 1,                        bias: True,                        all_poly:True,                        add_log:False,                        Acc:0.8037142120196028
Found a better scheme.                        Clean:1,                        Poly:1,                        Degree:5,                        lambda:1.1288378916846883e-09,                         Normalize: 1,                        bias: True,                        all_poly:True,                        add_log:False,                        Acc:0.8038818674232655
Found a better scheme.                        Clean:1,                        Poly:1,                        Degree:5,                        lambda:1e-10,                         Normalize: 1,                        bias: True,                        all_poly:True,               

In [8]:
max_acc = 0
lambdas = np.logspace(-10, 0, 10)
eval_func_name = 'ls'
test_func = implementations.least_squares
degrees = list(range(1,10))
for clean in [1,0]:
  for poly in [1,0]:
    if(poly):
      for degree in degrees:
        degree_list = [1] * 3
        degree_list[group_num] = degree
        for normalize in [1,0]:
          for add_bias in [True, False]:
            for n_sigma in [1.0, 3.0]:
              group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            data_utils.process_data(train_data, train_label, clean = clean)
              group0_x, group1_x, group2_x = \
                      data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly=True)
              cur_group = [group0_x, group1_x, group2_x]
              cur_labels = [group0_labels, group1_labels, group2_labels]
              group_x = cur_group[group_num]
              group_label = cur_labels[group_num]
              if(normalize):
                group_x = data_utils.normalize(group_x)
              if(add_bias):
                group_x = data_utils.add_bias(group_x)
              input_dict = {'lambda_':0,\
                      'initial_w':np.zeros(group_x.shape[1]),\
                      'max_iters':10000,\
                      'gamma':0.001,\
                      'sgd':False,\
                      'func_type':'linear'}
              loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict, eval_func_name)
 
              if(acc > max_acc):
                print(f'Found a better scheme.\
                    Clean:{clean},\
                    Poly:{poly},\
                    Degree:{degree},\
                    bias: {add_bias},\
                    Normalize: {normalize},\
                    all_poly:{True},\
                    n_sigma:{n_sigma},\
                    Acc:{acc}')
                max_acc = acc

Found a better scheme.                    Clean:1,                    Poly:1,                    Degree:1,                    bias: True,                    Normalize: 1,                    all_poly:True,                    n_sigma:1.0,                    Acc:0.7403639371381308
Found a better scheme.                    Clean:1,                    Poly:1,                    Degree:2,                    bias: True,                    Normalize: 1,                    all_poly:True,                    n_sigma:1.0,                    Acc:0.8204438930245382
Found a better scheme.                    Clean:1,                    Poly:1,                    Degree:3,                    bias: True,                    Normalize: 1,                    all_poly:True,                    n_sigma:1.0,                    Acc:0.8275847808105873
Found a better scheme.                    Clean:1,                    Poly:1,                    Degree:4,                    bias: True,                    Normal

In [ ]:
max_acc = 0
lambdas = np.logspace(-10, 0, 20)
# gammas = np.logspace(-4, -1, 10)
# gammas = [0.025,0.03,0.035,0.04]
gammas = [0.05, 0.075, 0.1]
# max_iters = [2000, 4000, 6000, 8000]
max_iters = [8000]
eval_func_name = 'logistic'
test_func = implementations.logistic_regression
degrees = [9,12,10,8]
for clean in [1]:
  for poly in [1]:
    if(poly):
      for degree in degrees:
        degree_list = [1] * 3
        degree_list[group_num] = degree
        for normalize in [1,0]:
          for all_poly in [True]:
            for add_bias in [True, False]:
              for n_sigma in [1.0, 3.0]:
                group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                              data_utils.process_data(train_data, train_label, clean = clean, sigma_n = n_sigma)
                group0_x, group1_x, group2_x = \
                        data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
                cur_group = [group0_x, group1_x, group2_x]
                cur_labels = [group0_labels, group1_labels, group2_labels]
                group_x = cur_group[group_num]
                group_label = cur_labels[group_num]
                group_label[group_label == -1] = 0
                if(normalize):
                  group_x = data_utils.normalize(group_x)
                if(add_bias):
                  group_x = data_utils.add_bias(group_x)
                for gamma in gammas:
                  for max_iter in max_iters:
                    input_dict = {'lambda_':0.00000001,\
                            'initial_w':np.zeros(group_x.shape[1]),\
                            'max_iters':max_iter,\
                            'gamma':gamma,\
                            'sgd':False,\
                            'func_type':'logistic'}
                    loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict, eval_func_name)
                    if(acc > max_acc):
                      print(f'Found a better scheme.\
                          Clean:{clean},\
                          Poly:{poly},\
                          Degree:{degree},\
                          gamma:{gamma}, \
                          max_iter:{max_iter}, \
                          bias: {add_bias},\
                          Normalize: {normalize},\
                          all_poly:{all_poly},\
                          n_sigma:{n_sigma},\
                          Acc:{acc}')
                      max_acc = acc

Found a better scheme.                      Clean:1,                      Poly:1,                      Degree:9,                      gamma:0.05,                       max_iter:8000,                       Normalize: 1,                      all_poly:True,                      Acc:0.8352417175457912
Found a better scheme.                      Clean:1,                      Poly:1,                      Degree:9,                      gamma:0.06,                       max_iter:8000,                       Normalize: 1,                      all_poly:True,                      Acc:0.8358222400160145


/content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1/implementations.py:10: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
/content/drive/MyDrive/EPFL_course_project/MLCS433/ML-Project1/implementations.py:199: RuntimeWarning: divide by zero encountered in log
  loss = -np.dot(y.T, np.log(g)) - np.dot((1 - y).T, np.log(1 - g))


KeyboardInterrupt: ignored